---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [164]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [165]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('/content/drive/MyDrive/spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [166]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [240]:
def answer_one():
    
    return (len(spam_data[spam_data.target==1])/len(spam_data))*100

In [241]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [242]:
from sklearn.feature_extraction.text import CountVectorizer
from operator import itemgetter

def answer_two():
    vect = CountVectorizer().fit(X_train)
    
    return sorted([(t, len(t)) for t in vect.vocabulary_.keys()], 
                  key=itemgetter(1))[-1][0]

In [243]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [171]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer()
    X_train_trans = vect.fit_transform(X_train)
    X_test_trans = vect.transform(X_test)

    mnb = MultinomialNB(alpha=0.1).fit(X_train_trans, y_train)
    y_hat = mnb.predict(X_test_trans)
    
    return roc_auc_score(y_test, y_hat)

In [172]:
answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [258]:
from sklearn.feature_extraction.text import TfidfVectorizer
from operator import itemgetter

def answer_four():
    tfidf = TfidfVectorizer()
    X_train_trans = tfidf.fit_transform(X_train)

    data = sorted([(feature, idf) for feature, idf in 
                                      zip(tfidf.get_feature_names_out(), 
                                          tfidf.idf_)], 
                                      key=itemgetter(1))
    smallest_tfidf = pd.Series([d[1] for d in data[:20]], index=[d[0] for d in data[:20]])
    largest_tfidf = pd.Series([d[1] for d in data[-21:-1]], index=[d[0] for d in data[-21:-1]])

    return (smallest_tfidf, largest_tfidf)

In [259]:
answer_four()

(to      2.198406
 you     2.265645
 the     2.707383
 in      2.890761
 and     2.976764
 is      3.003012
 me      3.111530
 for     3.206840
 it      3.222174
 my      3.231044
 call    3.297812
 your    3.300196
 of      3.319473
 have    3.354130
 that    3.408477
 on      3.463136
 now     3.465949
 can     3.545053
 are     3.560414
 so      3.566625
 dtype: float64, yuo         8.644919
 yuou        8.644919
 yupz        8.644919
 zac         8.644919
 zealand     8.644919
 zebra       8.644919
 zeros       8.644919
 zhong       8.644919
 zindgi      8.644919
 zoe         8.644919
 zouk        8.644919
 zyada       8.644919
 åè10        8.644919
 åòit        8.644919
 åôrents     8.644919
 ì¼1         8.644919
 ó_          8.644919
 û_thanks    8.644919
 ûï          8.644919
 ûïharry     8.644919
 dtype: float64)

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [175]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score


def answer_five():
    tfidf = TfidfVectorizer(min_df=3)
    X_train_trans = tfidf.fit_transform(X_train)
    X_test_trans = tfidf.transform(X_test)
    
    mnb = MultinomialNB(alpha=0.1).fit(X_train_trans, y_train)
    y_hat = mnb.predict(X_test_trans)
    
    return roc_auc_score(y_test, y_hat)

In [176]:
answer_five()

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [177]:
def answer_six():
    num_char = pd.DataFrame(spam_data["text"].apply(lambda x: len(x)).values, 
                        index=spam_data.target)
    avg = num_char.groupby("target").mean()
    
    return (avg.values[0][0], avg.values[1][0])

In [178]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [179]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [180]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score


def answer_seven():
    tfidf = TfidfVectorizer(min_df=5)
    X_train_trans = tfidf.fit_transform(X_train)
    X_test_trans = tfidf.transform(X_test)

    X_train_len = add_feature(X_train_trans, X_train.str.len())
    X_test_len = add_feature(X_test_trans, X_test.str.len())

    svm = SVC(C=10000).fit(X_train_len, y_train)
    y_hat = svm.predict(X_test_len)
    
    return roc_auc_score(y_test, y_hat)

In [181]:
answer_seven()

0.9661689557407943

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [196]:
def count_digits(text):
    count = 0
    for char in text:
        if char.isdigit():
            count += 1

    return count


def answer_eight():
    num_char = pd.DataFrame(spam_data["text"].apply(count_digits).values, 
                        index=spam_data.target)
    avg = num_char.groupby("target").mean()
    
    return (avg[0][0], avg[0][1])

In [197]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [210]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score


def answer_nine():
    tfidf = TfidfVectorizer(min_df=5, ngram_range=(1, 3))
    X_train_trans = tfidf.fit_transform(X_train)
    X_test_trans = tfidf.transform(X_test)
    
    X_train_final = add_feature(X_train_trans, [X_train.str.len(),
                                                X_train.apply(count_digits)])
    X_test_final = add_feature(X_test_trans, [X_test.str.len(),
                                              X_test.apply(count_digits)])
    
    log_reg = LogisticRegression(C=100, max_iter=1000).fit(X_train_final, y_train)
    y_hat = log_reg.predict(X_test_final)
    
    return roc_auc_score(y_test, y_hat)

In [211]:
answer_nine()

0.9678709064054463

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [207]:
def answer_ten():
    num_char = pd.DataFrame(spam_data["text"].str.count("\W").values, 
                        index=spam_data.target)

    avg = num_char.groupby("target").mean()

    return (avg[0][0], avg[0][1])

In [208]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [234]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


def answer_eleven():
    vect = CountVectorizer(min_df=5, ngram_range=(2, 5), analyzer="char_wb")
    X_train_trans = vect.fit_transform(X_train)
    X_test_trans = vect.transform(X_test)
    
    X_train_final = add_feature(X_train_trans, [X_train.str.len(),
                                                X_train.apply(count_digits), 
                                                X_train.str.count("\W")])
    X_test_final = add_feature(X_test_trans, [X_test.str.len(),
                                              X_test.apply(count_digits),
                                              X_test.str.count("\W")])
    
    log_reg = LogisticRegression(C=100, max_iter=1000).fit(X_train_final, y_train)
    y_hat = log_reg.predict(X_test_final)


    features = np.concatenate([vect.get_feature_names_out(), 
                                ['length_of_doc', 'digit_count', 
                                 'non_word_char_count']])
    coef_indx = log_reg.coef_[0].argsort()


    return (roc_auc_score(y_test, y_hat),
            features[coef_indx[:10]].tolist(),
            features[coef_indx[:-11:-1]].tolist())

In [235]:
answer_eleven()

(0.9780231906694056,
 [' i', 'ca', '..', '. ', 'pe', ' go', ' m', 'if', 'us', 'go'],
 ['digit_count', 'ia', ' r', 'xt', 'ne', 'co', ' ba', ' x', 'ian ', '46'])